In [ ]:
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
import wandb

wandb.login(key="WANDB_API_KEY")
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on Smart contracts vulnerabilities',
    job_type="training",
    anonymous="allow"
)

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

In [ ]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Analyze the given Solidity smart contract or vulnerability description carefully and identify potential security risks.
Before answering, think step-by-step and provide a logical and thorough analysis to ensure accuracy.

### Instruction:
You are a blockchain security expert specializing in Ethereum smart contracts, Solidity vulnerabilities, and DeFi exploits.
Your task is to analyze the provided smart contract or vulnerability description and identify possible security flaws.
Explain your reasoning clearly, referencing best practices and known exploits where applicable.

### Question:
{}

### Response:
<think>{}"""

In [ ]:
question = "What is the Write to Arbitrary Storage Location vulnerability?"


FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])

In [ ]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Analyze the given Solidity smart contract or vulnerability description carefully and identify potential security risks.
Before answering, think step-by-step and provide a logical and thorough analysis to ensure accuracy.

### Instruction:
You are a blockchain security expert specializing in Ethereum smart contracts, Solidity vulnerabilities, and DeFi exploits.
Your task is to analyze the provided smart contract or vulnerability description and identify possible security flaws.
Explain your reasoning clearly, referencing best practices and known exploits where applicable.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Input"]
    outputs = examples["Additional Context"]
    texts = []
    for input, cot, output in zip(inputs, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [ ]:
# Load the dataset
import pandas as pd
from datasets import Dataset

df = pd.read_csv("datasets/attacks_types.csv")

df["text"] = df["Input"] + "\n\n" + df["Additional Context"]

# Convert to a HF dataset
dataset = Dataset.from_pandas(df[["text"]])

dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
question = "What is the \"Write to Arbitrary Storage Location\" vulnerability?"


FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])

In [ ]:
new_model_local = "DeepSeek-R1-SC"
model.save_pretrained(new_model_local)
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

In [ ]:
from huggingface_hub import login
login()